In [1]:
#Importing packages
import numpy as np
import pandas as pd
import seaborn as sb
#import category_encoders as ce
import matplotlib.pyplot as plt
import pylab as pl
from pandas import ExcelFile
import xlrd
from sklearn import preprocessing
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn import metrics

In [2]:
#Openning Excel sheet and importing the data
"""
Open files and create the data frame from the excel sheet.
there are 24 columns and 142193 observations.
"""
#Assign spreadsheet file name to 'file'.
#file = 'Sample 1.xlsx'
file = 'WeatherAUS-Final-PCA-sample -2.xlsx'

#Load spreadsheet.
Excel_File = pd.ExcelFile (file)
print "The file name is :", file
print "The sheet's name is:", "Data"

#Load a sheet into a dataframe by name Data:
Data = Excel_File.parse('Data')
print "The matrix size is:", np.shape (Data)
print "The file has:", np.size (Data), "data points."
print "*******************************************"

#Data
X = Data.drop('label', axis=1)  
y = Data['label']
#print X.head()
#print y.head()

The file name is : WeatherAUS-Final-PCA-sample -2.xlsx
The sheet's name is: Data
The matrix size is: (2996, 9)
The file has: 26964 data points.
*******************************************


In [3]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(X, y)
X_sm = pd.DataFrame(X_sm)
y_sm = pd.DataFrame(y_sm)

#plot_2d_space(X_sm, y_sm, 'SMOTE over-sampling')
X_sm.head()

,0,1,2,3,4,5,6,7
0,-0.550179,3.875191,-1.487168,-0.618093,0.442401,0.396120,-0.033091,0.354812
1,-1.732818,0.616874,0.822514,-0.172073,0.923700,-0.527360,-0.010588,-0.160444
2,-2.014773,-0.489348,0.411100,0.892878,-0.020893,-0.324203,0.278403,0.542735
3,-1.317208,-0.076727,-0.457830,0.605717,0.122668,-0.318523,0.609186,0.625106
4,-0.639942,1.995414,0.273545,0.170543,-0.364265,-0.276448,0.660561,0.831699


In [10]:
X = X_sm
X.columns = ['P1', 'P2','P3','P4','P5','P6','P7','P8']
print X.head()
y = y_sm
y.columns = ['label']
print y.head()

         P1        P2        P3        P4        P5        P6        P7  \
0 -0.550179  3.875191 -1.487168 -0.618093  0.442401  0.396120 -0.033091   
1 -1.732818  0.616874  0.822514 -0.172073  0.923700 -0.527360 -0.010588   
2 -2.014773 -0.489348  0.411100  0.892878 -0.020893 -0.324203  0.278403   
3 -1.317208 -0.076727 -0.457830  0.605717  0.122668 -0.318523  0.609186   
4 -0.639942  1.995414  0.273545  0.170543 -0.364265 -0.276448  0.660561   

         P8  
0  0.354812  
1 -0.160444  
2  0.542735  
3  0.625106  
4  0.831699  
   label
0      1
1     -1
2     -1
3     -1
4     -1


In [ ]:
#Divide data into training and test sets
from sklearn.model_selection import train_test_split

#P stands for Primary sets 
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X, y, test_size = 0.20, random_state = 1) 

X_train = X_train_p
X_test = X_test_p
y_train = y_train_p
y_test = y_test_p 

X_train = pd.DataFrame(X_train)
X_train = X_train.reset_index()
X_train = X_train.drop('index', axis=1)                            
print X_train.shape

X_test = pd.DataFrame(X_test)
X_test = X_test.reset_index()
X_test = X_test.drop('index', axis=1)                            
print X_test.shape

y_train = pd.DataFrame(y_train)
y_train = y_train.reset_index()
y_train = y_train.drop('index', axis=1)

y_test = pd.DataFrame(y_test)
y_test = y_test.reset_index()
y_test = y_test.drop('index', axis=1)

# kernel SVM

In [ ]:
#Defining the kernels from Training Data
#Defining Linear Kernel
from numpy import linalg

X1 = X_train
X2 = pd.DataFrame.transpose(X_train)
#print X1.shape
#print X2.shape
K_Linear = np.dot(X1, X2)

print "The size of the matrix is:" ,np.shape(K_Linear)

#Creating dataframes
Kernel_1 = pd.DataFrame(K_Linear)

#Defining the kernels from Testing Data
#Defining Linear Kernel
from numpy import linalg

X1 = (X_train)
X2 = pd.DataFrame.transpose(X_test)
K_Linear = np.dot(X1, X2)

print "The size of the matrix is:" ,np.shape(K_Linear)

#Creating dataframes
Kernel_1_T = pd.DataFrame(K_Linear)

In [ ]:
#Optimization model
from gurobipy import *
model = Model ("Kernelizid multi-objective SVM")
model.params.presolve = 0

#Variables
A = {} #coefficients in kernelized format

S = {} #vector of deviations 

B = {} #off-set

#The counter in the dataset
Index = X_train.index 
print Index

#vtype= GRB.INTEGER, GRB.CONTINUOUS, GRB.BINARY    
for i in Index: 
    A[i] = model.addVar(vtype= GRB.CONTINUOUS ,lb= 0, ub= GRB.INFINITY)

for i in Index: 
    S[i] = model.addVar(vtype= GRB.CONTINUOUS ,lb= 0, ub= GRB.INFINITY)

for i in Index:
    B[i] = model.addVar(vtype= GRB.CONTINUOUS ,lb= 0 , ub= GRB.INFINITY)    
    
model.modelSense = GRB.MINIMIZE   #GRB.MAXIMIZE  
model.update()


for i in Index:
    model.addConstr ( (y_train.at[i,"label"]*(quicksum((A[j])*Kernel_1.at[i,j] for j in Index) + B[i])) >= (1 - S[i]))

Obj_Function = quicksum ( A[i] for i in Index)
Deviations = quicksum (S[i] for i in Index)
  
L = 0.9999

OBJ1 = L*Obj_Function
OBJ2 = (1-L)*Deviations    
objective = OBJ1 + OBJ2
        
model.setObjective (objective)
model.optimize()            

#Printing outputs 
if model.status==GRB.OPTIMAL:
    print "the model is optimal."
    print ("Optimal value:", model.objVal) 
    print "****************************************************"

In [ ]:
#Defining the hyperplaen
#Calculating the bias or off-set
B_list = []
for i in Index:
    B_list.extend([(B[i].x)])
Average = sum(B_list)/len(B_list)
Bias = Average 
#print Bias

#Calculating the weight vector
Weight_list = []
for i in Index:
    Weight_list.extend([(A[i].x)])
#print Weight_list

Weight_DF = pd.DataFrame(Weight_list)
Weight_DF.columns = ['Weight']
for i in Index:
    Y = y_train.at[i, 'label']*Weight_DF.at[i, 'Weight'] 
#Weight_DF = pd.DataFrame.transpose(Weight_DF)
#print Weight_DF
print np.shape(Weight_DF)

#Linear Kernel
print np.shape(Kernel_1_T)

Kernel_1_T = pd.DataFrame.transpose(Kernel_1_T)

X = np.dot(Kernel_1_T, Weight_DF)
#print X
X1 = pd.DataFrame(X, columns = ['X1'])

#H = Hyperplane
H = X1.add(Bias)
#print "The vector of hyperplane is:"
#print H
y_pred = []

#Testing
for i in H.index:
    if H.at[i,"X1"] > 0:
        y_pred.append(1)
    if H.at[i,"X1"] < 0:
        y_pred.append(-1)
    if H.at[i,"X1"] == 0:
        print i, "This point is a support vector."        
#print y_pred

In [ ]:
# Model Accuracy: how often is the classifier correct?
SVM_Linear_test_accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
SVM_Linear_test_precision = metrics.precision_score(y_test, y_pred)
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
SVM_Linear_test_recall = metrics.recall_score(y_test, y_pred)
print("Recall:",metrics.recall_score(y_test, y_pred))

#Model F-measure:Compute the F1 score, also known as balanced F-score or F-measure
F1 = 2 * (metrics.precision_score(y_test, y_pred) * metrics.recall_score
          (y_test, y_pred)) / (metrics.precision_score(y_test, y_pred) + metrics.recall_score(y_test, y_pred))
SVM_Linear_test_f1 = F1
print ('F-Measure:', F1)

#Model G-mean
SVM_Linear_test_Gmean  = geometric_mean_score(y_test, y_pred)
print ('Geometric mean:', geometric_mean_score(y_test, y_pred))
print "*******************************************"
#Evaluating the results
"""Confusion matrix, precision, recall, and F1 measures 
are the most commonly used metrics for classification tasks"""

target_names = ['class 0', 'class 1']
print(classification_report_imbalanced(y_test, y_pred,     target_names=target_names))
print "*******************************************"

#Confusion matrix using pyplot 
cm_SVC_linear = confusion_matrix(y_test, y_pred)
sb.set(style="whitegrid",font='sans-serif', font_scale=1.3)
ax= plt.subplot()
sb.heatmap(cm_SVC_linear, annot=True, ax = ax, cmap='coolwarm',fmt='g',linewidths=0.5 ) #annot=True to annotate cells

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_pred, y_test)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()